In [3]:
import requests
from bs4 import BeautifulSoup
import os
import csv

In [30]:
def extract_cars(url,main_div,estructura_1,estructura_2,imagen):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # clase_sup = soup.find_all("lo", {"class":'ui-search-layout ui-search-layout--grid'})
    divs = soup.find_all(main_div[0], {"class":main_div[1]})
    # print(divs)
    carros = []
    for item in divs:
        dict_list = {}
        clase_alta = BeautifulSoup(str(item),'html.parser')
        nombre_carro = clase_alta.find(estructura_1[0], class_=estructura_1[1]).text
        precio_carro = clase_alta.find(estructura_2[0], class_=estructura_2[1]).text
        imagenes = clase_alta.find_all(imagen[0], class_=imagen[1])
        for img in imagenes:
            url_imagen = img.get('src')
            if not url_imagen.endswith('.webp'):
                url_imagen = img.get('data-src')
            else:
                url_imagen = img.get('src')
        dict_list['Nombre'] = nombre_carro
        dict_list['Precio'] = precio_carro
        dict_list['Imagen_URL'] = url_imagen
        carros.append(dict_list)
    return carros

In [37]:
url = 'https://carros.tucarro.com.co/_Desde_529_NoIndex_True'
main_div = ['div','andes-card poly-card poly-card--grid-card andes-card--flat andes-card--padding-0 andes-card--animated']
estructura_1 = ['h2','poly-box poly-component__title']
estructura_2 = ['span','andes-money-amount__fraction']
imagen = ['img','poly-component__picture']
carros = extract_cars(url,main_div,estructura_1,estructura_2,imagen)

In [38]:
carros
with open('carros.csv', mode='a', newline='') as file: 
    writer = csv.writer(file)
    writer.writerow(['Nombre_Imagen', 'Precio'])
    file.seek(0, os.SEEK_END)  
    if file.tell() == 0:
        writer.writerow(['Nombre_Imagen', 'Precio']) 
    for item in carros:
        try:
            nombre_imagen = item['Nombre'].replace(' ', '_') + '.webp'
            ruta_imagen = os.path.join('imagenes_carros', nombre_imagen)
            response = requests.get(item['Imagen_URL'])
            response.raise_for_status()
            with open(ruta_imagen, 'wb') as f:
                f.write(response.content)
            writer.writerow([nombre_imagen, item['Precio']])
        except Exception as e:
            continue

In [53]:
def extract_cars_2(url,main_div,estructura_1,estructura_2,img_src):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # clase_sup = soup.find_all("lo", {"class":'ui-search-layout ui-search-layout--grid'})
    divs = soup.find_all(main_div[0], {"class":main_div[1]})
    # print(divs)
    carros = []
    for item in divs:
        dict_list = {}
        clase_alta = BeautifulSoup(str(item),'html.parser')
        nombre_carro = clase_alta.find(estructura_1[0], itemprop=estructura_1[1]).text
        precio_carro = clase_alta.find(estructura_2[0], class_=estructura_2[1]).text
        imagenes = clase_alta.find_all(img_src[0], itemprop_=img_src[1])
        url_imagen = ''
        for img in imagenes:
            url_imagen = img.get('src')
            if not url_imagen.endswith('.webp'):
                url_imagen = img.get('data-src')
            else:
                url_imagen = img.get('src')
        dict_list['Nombre'] = nombre_carro
        dict_list['Precio'] = precio_carro
        dict_list['Imagen_URL'] = url_imagen
        carros.append(dict_list)
    return carros

In [54]:
url = 'https://www.autocosmos.com.co/catalogo/vigente/audi'
main_div = ['section','section m-brand']
estructura_1 = ['meta','model']
estructura_2 = ['span','model-card__price']
img_src = ['img','image']
carros_brand = extract_cars_2(url,main_div,estructura_1,estructura_2,img_src)

In [55]:
carros_brand

[{'Nombre': '',
  'Precio': '\n\n\n\nPrecio de lista a partir de:$155.400.000\n',
  'Imagen_URL': ''}]

In [44]:
carros_brand
with open('carros_2.csv', mode='a', newline='') as file: 
    writer = csv.writer(file)
    writer.writerow(['Nombre_Imagen', 'Precio'])
    file.seek(0, os.SEEK_END)  
    if file.tell() == 0:
        writer.writerow(['Nombre_Imagen', 'Precio']) 
    for item in carros_brand:
        try:
            nombre_imagen = item['Nombre'].replace(' ', '_') + '.webp'
            ruta_imagen = os.path.join('car_brand', nombre_imagen)
            response = requests.get(item['Imagen_URL'])
            response.raise_for_status()
            with open(ruta_imagen, 'wb') as f:
                f.write(response.content)
            writer.writerow([nombre_imagen, item['Precio']])
        except Exception as e:
            continue